In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from itertools import combinations, permutations
from matplotlib import pyplot as plt
from scipy.stats import entropy
from scipy.special import kl_div
from utils import get_data_train

In [ ]:
df = get_data_train()
n_bins = 5 # tu możemy wybrać różną liczbę kuełków go generowania histogramów
            # liczbę przedziałów na jakich będziemy liczyć dywergencję

In [ ]:
activities = np.unique(df['activity'])
activities_dict = {activity: None for activity in activities}
activities_dict

In [ ]:
for activity in activities_dict.keys():
    numerical_features = df[df['activity']==activity].drop(['activity', 'subject'], axis='columns')
    activities_dict[activity] = numerical_features.apply(
        lambda column: 
        np.histogram(column, bins=n_bins, density=True, range=(-1,1))[0])

Bierzemy permutacje cech, bo kl_div nie jest przemienna

In [ ]:
combi_colnames = list(map('-'.join, list(permutations(activities,2))))
combi_df = pd.DataFrame(columns = combi_colnames)

numerical_features = df.drop(['activity', 'subject'], axis='columns').columns
for feature in numerical_features:
    combi_df = combi_df.append(pd.Series(np.repeat(feature, 30), name=feature, index=combi_colnames))

In [ ]:
def replace_inf_and_sum(feature1, feature2, x):
    kl = kl_div(
        activities_dict[feature1][x],
        activities_dict[feature2][x])
    return sum(map(lambda x: 100 if x>100 else x, kl))

def count_inf(feature1, feature2, x):
    return sum(
        np.isinf(          # tu zliczamy infy
            kl_div(
                activities_dict[feature1][x],
                activities_dict[feature2][x])))

def take_median(feature1, feature2, x):
    return np.median(          
            kl_div(
                activities_dict[feature1][x],
                activities_dict[feature2][x]))

In [ ]:
def apply_to_column(column):
    feature1, feature2 = column.name.split('-')
    return column.apply(lambda x: replace_inf_and_sum(feature1, feature2, x)) # w tej lambdzie można wybrać inną funkcję


result = combi_df.apply(apply_to_column)

#### Jak agregować wyniki w `apply_to_column`?

Teraz liczymy ile razy było infinity, czyli w ilu miejscach rozkłady bardzo się różniły.

Co można innego? 
- może można zamienić inf na jakiś duży threshold, np 100, i liczyć sumę lub średnią
- można liczyć medianę

#### Co mamy?

Ramkę danych, wiersze to cechy, kolumny to dwuelementowa kombinacja aktywności. Wartości to miara jak bardzo różnią się rozkłady cech dla dwóch aktywności.

#### Jak agregować wyniki do poziomu cechy?

- bierzemy max z wiersza, otrzymamy kolumny ze zbioru danych, które najlepiej **rozdzieliły jakieś dwie wybrane aktywności**
- bierzemy sumę/średnią, otrzymamy kolumny ze bioru danych, które **średnio najlepiej rozdzielają** aktywności


Chyba lepiej wybierać metodą z max, wtedy otrzymamy rozdzielenie różnych rozkładów.

In [ ]:
chosen_best = result.apply(lambda x: max(x), axis=1).sort_values(ascending=False).head(20).index.to_list()
avg_best = result.apply(lambda x: sum(x), axis=1).sort_values(ascending=False).head(20).index.to_list()

In [ ]:
def plot_var(varname):
    ax = sns.displot(df, x=varname, hue='activity', kind='kde', log_scale=(False, True))#bins=n_bins, multiple='dodge')
    plt.show()

In [ ]:
for varname in avg_best: # lub alternatywnie chosen_best
    plot_var(varname)